<font size="6"> **Multifactor Risk Models** </font>

In [54]:
#import cvxpy as cvx
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

import pickle

from mle_quant_utils import quant_helper, quant_factors


%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

In [2]:
from zipline.data import bundles
from zipline.utils.calendars import get_calendar
from zipline.pipeline.factors import AverageDollarVolume
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import Returns, SimpleMovingAverage
from zipline.pipeline.data import USEquityPricing

from zipline.pipeline.factors import CustomFactor, DailyReturns, Returns, SimpleMovingAverage, AnnualizedVolatility
from zipline.pipeline.data import USEquityPricing

from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar


from zipline.data.data_portal import DataPortal

In [18]:
import yaml
import os

# Retrieve parameters from configuration file
with open("../conf.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)

BUNDLE_FOLDER = cfg['quantopian']['dataset2']['bundle_folder']
BUNDLE_NAME = cfg['quantopian']['dataset2']['bundle_name']
SECTOR_FOLDER = cfg['quantopian']['dataset2']['sector_folder']
SECTOR_DATA = cfg['quantopian']['dataset2']['sector_data']
SECTOR_NAMES = cfg['quantopian']['dataset2']['sector_names']

# Specify the bundle path
bundle_path = os.path.join(os.getcwd(), '..', 'data', BUNDLE_FOLDER)
sector_path = os.path.join(os.getcwd(), '..', 'data', SECTOR_FOLDER, SECTOR_DATA)
sector_file = os.path.join(os.getcwd(), '..', 'data', SECTOR_FOLDER, SECTOR_NAMES)
os.path.isdir(bundle_path)

True

In [22]:
MODEL_VERSION = 'v01'
print('Model version: ', MODEL_VERSION)

Model version:  v01


In [23]:
INPATH =  "../data/" + cfg['output']['main'] + "/" + cfg['output']['alpha_factors_ml']['folder'] + "/" + MODEL_VERSION +"/"
INFILE1 = cfg['output']['alpha_factors_ml']['features']
INFILE2 = cfg['output']['alpha_factors_ml']['targets']
INFILE3 = cfg['output']['alpha_factors_ml']['model']

In [25]:
split_col = cfg['models']['alpha_ml']['split_col']

# Load Data

## Data Bundle

In [4]:
adv_win = cfg['models']['universe']['window']
adv_top = cfg['models']['universe']['adv_top']
print('Universe ADV window: {} and top threshold: {}'.format(adv_win, adv_top))

Universe ADV window: 120 and top threshold: 500


In [5]:
# Data Bundle
os.environ['ZIPLINE_ROOT'] = bundle_path
ingest_func = bundles.csvdir.csvdir_equities(['daily'], BUNDLE_NAME)
bundles.register(BUNDLE_NAME, ingest_func)
print('Data Registered')

Data Registered


In [6]:
universe = AverageDollarVolume(window_length=adv_win).top(adv_top) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(BUNDLE_NAME)
engine =  quant_helper.build_pipeline_engine(bundle_data, trading_calendar)

In [63]:
universe_end_date =  pd.Timestamp( cfg['models']['universe']['start'], tz='UTC') # pd.Timestamp('2016-01-05', tz='UTC')
universe_start_date =  pd.Timestamp( cfg['models']['universe']['end_risk'], tz='UTC')  # universe_end_date - pd.DateOffset(years=5)
print('Universe start: {} and end: {} dates'.format(universe_end_date, universe_start_date))

Universe start: 2016-01-05 00:00:00+00:00 and end: 2011-01-05 00:00:00+00:00 dates


In [8]:
universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers[:5]

[Equity(0 [A]),
 Equity(1 [AAL]),
 Equity(2 [AAP]),
 Equity(3 [AAPL]),
 Equity(4 [ABBV])]

## Data Portal
Not that we have our pipeline built, let's access the returns data. We'll start by building a data portal.

In [9]:
data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

In [10]:
pricing = quant_helper.get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_start_date,
        universe_end_date)
   
five_year_returns = pricing .pct_change()[1:].fillna(0)
pd.concat([five_year_returns.head(2), five_year_returns.tail(2)],axis=0)

C:\Users\manua\Documents\repos\dslab_ai4trading\risk_factors\mle_quant_utils\quant_helper.py:54: FutureWarning: offset is deprecated. Use freq instead
  end_dt = pd.Timestamp(end_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')
C:\Users\manua\Documents\repos\dslab_ai4trading\risk_factors\mle_quant_utils\quant_helper.py:55: FutureWarning: offset is deprecated. Use freq instead
  start_dt = pd.Timestamp(start_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1256 entries, 2011-01-07 to 2016-01-05
Freq: C
Columns: 490 entries, Equity(0 [A]) to Equity(490 [ZTS])
dtypes: float64(490)
memory usage: 4.7 MB


## ML-Alpha

In [26]:
features_df = pd.read_csv(INPATH + INFILE1, parse_dates=['date'])
features_df['date'] = features_df['date'].dt.tz_localize('utc')
features_df.set_index(['date', 'asset'], inplace=True)
split = features_df.pop(split_col)
features = features_df.columns.tolist()
pd.concat([features_df.head(2), features_df.tail(2)],axis=0)

Mean_Reversion_Sector_Neutral_Smoothed  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                                   -0.795709   
                          AAL                                  0.558847   
2015-12-28 00:00:00+00:00 ZION                                 0.773366   
                          ZTS                                 -0.944433   

                                 Momentum_1YR_Smoothed  \
date                      asset                          
2013-01-03 00:00:00+00:00 A                  -1.230279   
                          AAL                 1.713471   
2015-12-28 00:00:00+00:00 ZION               -0.188031   
                          ZTS                 0.209727   

                                 Overnight_Sentiment_Smoothed  adv_120d  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                         -1.301799  1.338573   
                          AAL                       -1.632765  1.139994   
2015-12-28 00:00:00+00:00 ZION                       1.325552 -0.965002   
                          ZTS                        0.441851  0.229762   

                                  adv_20d  dispersion_120d  dispersion_20d  \
date                      asset                                              
2013-01-03 00:00:00+00:00 A      1.397411         0.013270        0.011178   
                          AAL    1.081155         0.013270        0.011178   
2015-12-28 00:00:00+00:00 ZION  -0.724635         0.014916        0.014731   
                          ZTS   -0.180275         0.014916        0.014731   

                                 market_vol_120d  market_vol_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A             0.127654        0.135452   
                          AAL           0.127654        0.135452   
2015-12-28 00:00:00+00:00 ZION          0.183680        0.181479   
                          ZTS           0.183680        0.181479   

                                 volatility_120d  ...    sector_code_5  \
date                      asset                   ...                    
2013-01-03 00:00:00+00:00 A            -0.836546  ...                0   
                          AAL           1.639924  ...                0   
2015-12-28 00:00:00+00:00 ZION          0.547895  ...                1   
                          ZTS           0.413572  ...                0   

                                 sector_code_6  sector_code_7  sector_code_8  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0              0              0   
                          AAL                0              0              0   
2015-12-28 00:00:00+00:00 ZION               0              0              0   
                          ZTS                0              0              0   

                                 sector_code_9  sector_code_10  qtr_1  qtr_2  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0               0      1      0   
                          AAL                0               0      1      0   
2015-12-28 00:00:00+00:00 ZION               0               0      0      0   
                          ZTS                0               0      0      0   

                                 qtr_3  qtr_4  
date                      asset                
2013-01-03 00:00:00+00:00 A          0      0  
                          AAL        0      0  
2015-12-28 00:00:00+00:00 ZION       0      1  
                          ZTS        0      1  

[4 rows x 33 columns]

In [9]:
valid_dates = split[split=='valid'].index.get_level_values(0)
valid_start = valid_dates[0].strftime('%Y-%m')
valid_end = valid_dates[-1].strftime('%Y-%m')

In [10]:
test_dates = split[split=='test'].index.get_level_values(0)
test_start = test_dates[0].strftime('%Y-%m')
test_end = test_dates[-1].strftime('%Y-%m')

In [41]:
train_valid_idx = split[(split=='train') | (split=='valid')].index.get_level_values(0).drop_duplicates()

In [51]:
X_train_valid = features_df.loc[(train_valid_idx, slice(None)), :]
pd.concat([X_train_valid.head(2), X_train_valid.tail(2)],axis=0)

Mean_Reversion_Sector_Neutral_Smoothed  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                                   -0.795709   
                          AAL                                  0.558847   
2015-05-27 00:00:00+00:00 ZION                                 0.216057   
                          ZTS                                 -1.418772   

                                 Momentum_1YR_Smoothed  \
date                      asset                          
2013-01-03 00:00:00+00:00 A                  -1.230279   
                          AAL                 1.713471   
2015-05-27 00:00:00+00:00 ZION               -0.884221   
                          ZTS                 1.393649   

                                 Overnight_Sentiment_Smoothed  adv_120d  \
date                      asset                                           
2013-01-03 00:00:00+00:00 A                         -1.301799  1.338573   
                          AAL                       -1.632765  1.139994   
2015-05-27 00:00:00+00:00 ZION                      -0.078567 -1.085659   
                          ZTS                        1.378501  0.249987   

                                  adv_20d  dispersion_120d  dispersion_20d  \
date                      asset                                              
2013-01-03 00:00:00+00:00 A      1.397411         0.013270        0.011178   
                          AAL    1.081155         0.013270        0.011178   
2015-05-27 00:00:00+00:00 ZION  -1.342789         0.012559        0.011522   
                          ZTS    0.378552         0.012559        0.011522   

                                 market_vol_120d  market_vol_20d  \
date                      asset                                    
2013-01-03 00:00:00+00:00 A             0.127654        0.135452   
                          AAL           0.127654        0.135452   
2015-05-27 00:00:00+00:00 ZION          0.129420        0.104706   
                          ZTS           0.129420        0.104706   

                                 volatility_120d  ...    sector_code_5  \
date                      asset                   ...                    
2013-01-03 00:00:00+00:00 A            -0.836546  ...                0   
                          AAL           1.639924  ...                0   
2015-05-27 00:00:00+00:00 ZION          0.664252  ...                1   
                          ZTS          -0.321412  ...                0   

                                 sector_code_6  sector_code_7  sector_code_8  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0              0              0   
                          AAL                0              0              0   
2015-05-27 00:00:00+00:00 ZION               0              0              0   
                          ZTS                0              0              0   

                                 sector_code_9  sector_code_10  qtr_1  qtr_2  \
date                      asset                                                
2013-01-03 00:00:00+00:00 A                  0               0      1      0   
                          AAL                0               0      1      0   
2015-05-27 00:00:00+00:00 ZION               0               0      0      1   
                          ZTS                0               0      0      1   

                                 qtr_3  qtr_4  
date                      asset                
2013-01-03 00:00:00+00:00 A          0      0  
                          AAL        0      0  
2015-05-27 00:00:00+00:00 ZION       0      0  
                          ZTS        0      0  

[4 rows x 33 columns]

In [55]:
with open(INPATH + INFILE3, 'rb') as file:
    model = pickle.load(file)

In [62]:
ml_alpha = pd.Series(index=features_df.index, data=model.predict(features_df))
ml_alpha[split_col] = split